In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
! pip install --upgrade efficientnet-pytorch

Requirement already up-to-date: efficientnet-pytorch in /usr/local/lib/python3.6/dist-packages (0.6.3)


In [2]:
import torch
import numpy as np
import torch
from torch.utils import data

# Import dataset related API
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import models
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Import common neural network API in pytorch
import torch.nn as nn
import torch.nn.functional as F

# Import optimizer related API
import torch.optim as optim

# Import Efficient net model (pytorch implementation)
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

# Fix the random seed of pytorch related function
torch.manual_seed(0)

# Fix the random seed of numpy related function
np.random.seed(0) 

# Check device, using gpu 0 if gpu exist else using cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
labeled_data = torch.Tensor(np.load('drive/My Drive/f2019-aihw7/mnist-train-images.npy'))
labels = torch.Tensor(np.load('drive/My Drive/f2019-aihw7/mnist-train-labels.npy'))
val_data = torch.Tensor(np.load('drive/My Drive/f2019-aihw7/mnist-val-images.npy'))
val_labels = torch.Tensor(np.load('drive/My Drive/f2019-aihw7/mnist-val-labels.npy'))

None of the transforms in the first teacher model should have any noise.

In [4]:
transform = transforms.Compose([transforms.Grayscale(3),transforms.Scale(32),transforms.ToTensor()])

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [0]:
i = 0
label_data = []
while i<60000:
  label_data.append(transform(transforms.ToPILImage()(labeled_data[i])).unsqueeze(0))
  i+=1

In [0]:
i = 0
v_data = []
while i<10000:
  v_data.append(transform(transforms.ToPILImage()(val_data[i])).unsqueeze(0))
  i+=1

In [0]:
label_data = torch.cat(label_data)
v_data = torch.cat(v_data)

In [0]:
my_dataset = data.TensorDataset(label_data,labels) 
my_dataloader = data.DataLoader(my_dataset,batch_size=248) 

In [0]:
val_dataset = data.TensorDataset(v_data,val_labels) 
val_dataloader = data.DataLoader(val_dataset,batch_size=248) 

In [0]:
model = EfficientNet.from_name('efficientnet-b0')
#model = EfficientNet.from_pretrained('efficientnet-b0')

In [0]:
# Unfreeze model weights
for param in model.parameters():
    param.requires_grad = True

In [0]:
num_ftrs = model._fc.in_features

model._fc = nn.Sequential(nn.Linear(num_ftrs, 10),
                         nn.Softmax(1))
model = model.to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#optimizer = optim.Adam(model.parameters())
#loss_func = nn.BCELoss()
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [19]:
print('Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy')
print('____________________________________________________________________')
q=0
for epoch in range(5):
    running_loss = 0.0
    running_corrects = 0
    model.train()
    for i, data in enumerate(my_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.to(device)
            labels = labels.to(device)


        # zero the parameter gradients: Clean the gradient caclulated in the previous iteration
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())

        # Calculate gradient of matrix with requires_grad = True
        loss.backward()

        # Apply the gradient calculate from last step to the matrix
        optimizer.step()
        # Add 1 more iteration count to learning rate scheduler
        #scheduler.step()

        # print statistics
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

    val_corrects = 0
    val_loss = 0
    model.eval()

    with torch.no_grad():
        for data in val_dataloader:
            images, labels = data
            if torch.cuda.is_available():
                images = images.to(device)
                labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_loss += criterion(outputs, labels.long())
            num = torch.sum(preds == labels.data)
            val_corrects += num
    #print('Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy')

    print("  %2d   |    %.4f   |     %.4f       |   %.4f   |     %.4f    " 
          %(q,running_loss/60000,int(running_corrects)/60000,val_loss/10000,int(val_corrects)/10000 ))

    q+=1




print('Finished Training')

Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy
____________________________________________________________________
  15   |    0.0061   |     0.9549       |   0.0062   |     0.9465    
  16   |    0.0061   |     0.9577       |   0.0062   |     0.9481    
  17   |    0.0061   |     0.9608       |   0.0062   |     0.9480    
  18   |    0.0061   |     0.9629       |   0.0062   |     0.9511    
  19   |    0.0061   |     0.9644       |   0.0062   |     0.9506    
Finished Training
